In [ ]:
import openai
from openai import OpenAI
from PIL import Image
import io
import os
import shutil
import base64
import csv
from tqdm import tqdm

In [ ]:
api_key = "..."

In [ ]:
def prepare_example_prompt(example_image_path, example_summary):
    # Load the example image
    with open(example_image_path, "rb") as img_file:
        example_image = img_file.read()

    # Format the prompt with the example image and summary
    example_prompt = (
        "Here is an example of an image and its summary:\n\n"
        "Image: [Example image provided below]\n"
        f"Summary: {example_summary}\n\n"
        "Now, analyze this new image in the same format:\n"
        "Image: [New image provided below]\n"
        "Summary:"
    )

    return example_prompt, example_image

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def generate_summary_for_image(image_path, prompt):
    base64_image = encode_image(image_path)

    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides building condition annotations."},
            {"role": "user", "content": [
                {"type": "text", "text": f"{prompt}"},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ]}
        ],
#         max_tokens=300,
    )

    return response.choices[0].message.content

In [ ]:
def generate_contrast_for_image(image_path_before, image_path_after, prompt):
    base64_image_before = encode_image(image_path_before)
    base64_image_after = encode_image(image_path_after)

    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides hurricane condition annotations."},
            {"role": "user", "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image_before}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image_after}"}},
                {"type": "text", "text": f"{prompt}"},
            ]}
        ],
#         max_tokens=300,
    )

    return response.choices[0].message.content

In [ ]:
def classify_image(image_path_before, image_path_after, prompt, prompt_summary, prompt_contrast):
    base64_image_before = encode_image(image_path_before)
    base64_image_after = encode_image(image_path_after)

    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides hurricane condition annotations."},
            {"role": "user", "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image_before}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image_after}"}},
                {"type": "text", "text": f"{prompt_summary[0]}"},
                {"type": "text", "text": f"{prompt_summary[1]}"},
                {"type": "text", "text": f"{prompt_summary[2]}"},
                {"type": "text", "text": f"{prompt_summary[3]}"},
                {"type": "text", "text": f"{prompt_contrast}"},
                {"type": "text", "text": f"{prompt}"},
            ]}
        ],
#         max_tokens=300,
    )

    return response.choices[0].message.content

In [ ]:
def process_response(response):
    lines = response.strip().split("\n")

    results = {}

    for line in lines:
        if 'score:' in line:
            description, score = line.split('score:')
            description = description.strip()
            score_value = int(score.split('/')[0].strip())
            results[description] = score_value

        elif 'degree:' in line:
            description, degree = line.split('degree:')
            description = description.strip()
            degree_value = degree.strip()

            if degree_value == 'minor damage':
                degree_value = "Minor"
            elif degree_value == 'moderate damage':
                degree_value = "Moderate"
            elif degree_value == 'severe damage':
                degree_value = "Severe"

            results[description] = degree_value

    return results

In [ ]:
if __name__ == '__main__':
#     base_dir = './datasets/matched_image_pairs_base'
#     annotated_dir = './annotated_datasets/'
#     csv_file_path = './image_summaries.csv'
    base_dirs = ['./final_pair_label_classified_folders/folder_0/',
                './final_pair_label_classified_folders/folder_1/',
                './final_pair_label_classified_folders/folder_2/']
    csv_file_name = 'image_summaries.csv'

    for base_dir in base_dirs:
        csv_file_path = base_dir + csv_file_name
        with open(csv_file_path, mode='w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["degree","score","trees","trash","building","water","image_path","year","root","summary","contrast"])


            # Traverse each subfolder and collect image paths
    #         for root, _, files in os.walk(base_dir):
            for root, _, files in tqdm(os.walk(base_dir), desc="Processing Folders"):
                files = [file for file in files if file.endswith('.png')]

                for i in range(0, len(files), 2):
                    if i + 1 < len(files):  # Ensure there are pairs of images
                        file_1, file_2 = files[i], files[i + 1]

                        image_path_1 = os.path.join(root, file_1)
                        image_path_2 = os.path.join(root, file_2)

                        year_1 = image_path_1[-8:-4] if image_path_1[-4:].lower() == '.png' else image_path_1[-4:]
                        year_2 = image_path_2[-8:-4] if image_path_2[-4:].lower() == '.png' else image_path_2[-4:]

                        if year_1.isdigit() and year_2.isdigit():
                            if year_1 == "2023":
                                file_before = file_1
                                file_after = file_2
                                year_before = year_1
                                year_after = year_2
                                image_path_before = image_path_1
                                image_path_after = image_path_2
                            elif year_1 == "2024":
                                file_before = file_2
                                file_after = file_1
                                year_before = year_2
                                year_after = year_1
                                image_path_before = image_path_2
                                image_path_after = image_path_1

                        prompts = [
                            "Based on fallen trees, describe the damage situation in 50 words.",
                            "Based on housing trash, describe the damage situation in 50 words.",
                            "Based on street signs or destroyed buildings, describe the damage situation in 50 words.",
                            "Based on standing water in the street, describe the damage situation in 50 words.",
                            "The first image is a photo before the disaster, and the second is a photo after the disaster, depicting the disaster and the changes after the disaster according to the content of the picture in 50 words."
                        ]

                        summary_after = []
                        summary_tree = generate_summary_for_image(image_path_after, prompts[0])
                        summary_trash = generate_summary_for_image(image_path_after, prompts[1])
                        summary_building = generate_summary_for_image(image_path_after, prompts[2])
                        summary_water = generate_summary_for_image(image_path_after, prompts[3])
                        summary_after.append(summary_tree)
                        summary_after.append(summary_trash)
                        summary_after.append(summary_building)
                        summary_after.append(summary_water)
    #                     print(f"Summary: {summary_after}")

                        summary_contrast = generate_contrast_for_image(image_path_before, image_path_after, prompts[-1])
    #                     print(f"Summary: {summary_contrast}")

                        prompt = "The first picture is a photo before the disaster, and the second picture is after the disaster, \
                        and according to the content and text in the picture, \
                        the disasters that caused these changes are divided into three categories: minordamage, moderate damage, and severedamage. \
                        light damage images are characterized by a clean scene with no significant damage or only light damage, such as small areas of fallen trees or a few small road signs knocked down. \
                        Medium damage images are relatively cluttered and typically include larger or more extensive areas of fallen trees, as well as standing water around the trees. \
                        These images may also show more fallen road signs or road closure signs. \
                        Heavy damage images are very chaotic, featuring large or extensive areas of fallen trees, flooded roads, and housing trash. \
                        The output is one of these three categories for the second image. \
                        I dont need any explanation. \
                        The output format is as follows: \
                        fallen trees score: [fallen trees score / 100] \
                        housing trash score: [housing trash score / 100] \
                        destroy buildings score: [destroy buildings score / 100] \
                        standing water in the street score: [standing water in the street score / 100] \
                        damage degree score: [damage degree score / 100] \
                        damage degree: [minor damage, moderate damage, and severe damage]"
                        response = classify_image(image_path_before, image_path_after, prompt, summary_after, summary_contrast)
                        score = process_response(response)

                        normalized_root = root.replace('\\', '/')
                        writer.writerow([score['damage'], score['damage degree'], score['fallen trees'], score['housing trash'], score['destroy buildings'], score['standing water in the street'], file_after[:-4], year_after, normalized_root, summary_after, summary_contrast])
    #                     relative_path = os.path.relpath(image_path, base_dir)
    #                     target_path = os.path.join(annotated_dir, relative_path)
    #                     os.makedirs(os.path.dirname(target_path), exist_ok=True)

                        # shutil.copy(image_path, target_path)

    print(f"CSV file created at {csv_file_path}")

Processing Folders: 2it [00:26, 13.02s/it]
Processing Folders: 2it [00:18,  9.27s/it]
Processing Folders: 2it [00:25, 12.89s/it]

CSV file created at ./final_pair_label_classified_folders/folder_2/image_summaries.csv
